# EFM First-Principles Simulation: The Cosmogenesis Model (V9 - Structure Formation)

## Objective: Simulate the Emergence of Stable Solitons from the EFM Vacuum

This simulation builds directly upon the success of the `CosmogenesisV8` model, which demonstrated the formation of a stable S/T vacuum from a tachyonic inflationary phase. The key innovation in V9 is the implementation of **density-dependent physical laws**, a core EFM principle derived from analyzing the parameter differences between cosmological and particle states.

**The V9 Cosmological Sequence:**
1.  **Epoch 1: Tachyonic Inflation (`t < 1000`):** Identical to V8. The simulation begins with an unstable potential (`m² < 0`) to inflate the vacuum from a random noise field.
2.  **Epoch 2: The Grand Unification Transition (`t = 1000`):** Identical to V8. The potential becomes fundamentally stable.
3.  **Epoch 3: Density-Dependent Evolution (`t > 1000`):** This is the crucial new phase. The NLKG equation's parameters (`m²` and `g`) are no longer constant. They are calculated dynamically at every point in space based on the local field density `ρ`.
    - **In low-density regions (vacuum):** The parameters revert to the stable S/T (cosmological) state, maintaining the vacuum.
    - **In high-density regions (fluctuations):** The parameters switch to the S=T (particle) state, introducing an attractive nonlinearity (`g < 0`) and a strong mass term (`m² = 1.0`). This creates a potential well, hypothesized to trap the energy and form a stable, localized soliton.

This simulation tests the core EFM hypothesis that particles are self-organizing, stable states that emerge when field density crosses a critical threshold, thereby locally altering the fabric of physical law.

In [ ]:
import os
import torch
import torch.nn.functional as F
import gc
from tqdm.notebook import tqdm
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import glob

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not in Google Colab environment.")

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(f"Using GPU: {torch.cuda.get_device_name(device)}, VRAM: {torch.cuda.get_device_properties(device).total_memory / 1e9:.2f} GB")
else:
    device = torch.device('cpu')
    print("No GPU available, running on CPU.")

data_path_dynamic = '/content/drive/My Drive/EFM_Simulations/data/FirstPrinciples_Dynamic_N512_v9_StructureFormation/'
os.makedirs(data_path_dynamic, exist_ok=True)
print(f"Structure Formation (V9) Data will be saved to: {data_path_dynamic}")

In [ ]:
config = {
    'N': 512,
    'L_sim_unit': 40.0,
    'T_steps': 100000,
    'dt_cfl_factor': 0.001,
    'c_sim_unit': 1.0,

    # --- Cosmogenesis Parameters ---
    'transition_step': 1000,       # Step to end inflation
    'm_sq_inflation': -0.25,       # Tachyonic potential

    # --- Density-Dependent Physics Parameters (The Core of V9) ---
    'rho_threshold': 0.0005,       # Critical density to switch physics
    'k_density_coupling': 0.01,    # k for ρ = k * φ²

    # Parameters for S/T state (Vacuum)
    'm_sq_vacuum': 0.01,           # Low mass term, from Dimensionless Params paper (m=0.1)
    'g_vacuum': 0.1,               # Repulsive nonlinearity, from Dimensionless Params paper

    # Parameters for S=T state (Particle)
    'm_sq_particle': 1.0,          # Strong mass term for confinement, from Mass Gen paper
    'g_particle': -0.1,            # Attractive nonlinearity for potential well, from Mass Gen paper

    # Universal parameters (assumed constant for simplicity)
    'alpha_structure': 0.7,        # From Dimensionless Params paper
    'eta_sim': 0.01,               # From Dimensionless Params paper
    'delta_sim': 0.0002,           # From Dimensionless Params paper

    # Stability & Initial Conditions
    'sponge_width': 0.1,
    'sponge_strength': 0.99,
    'initial_noise_amplitude': 1.0e-4,

    'history_every_n_steps': 500,
    'checkpoint_every_n_steps': 10000
}

config['dx_sim_unit'] = config['L_sim_unit'] / config['N']
config['dt_sim_unit'] = config['dt_cfl_factor'] * config['dx_sim_unit'] / config['c_sim_unit']
config['run_id'] = f"DynamicPhysics_N{config['N']}_T{config['T_steps']}_StructureV9"

print(f"--- EFM Structure Formation Configuration (V9) ---")
for key, value in config.items():
    print(f"{key}: {value}")

In [ ]:
# --- JIT-COMPILED SIMULATION FUNCTIONS (v9) --- #

@torch.jit.script
def conv_laplacian_gpu(phi_field: torch.Tensor, dx: float) -> torch.Tensor:
    stencil = torch.tensor([[[0.,0.,0.],[0.,1.,0.],[0.,0.,0.]],[[0.,1.,0.],[1.,-6.,1.],[0.,1.,0.]],[[0.,0.,0.],[0.,1.,0.],[0.,0.,0.]]], dtype=phi_field.dtype, device=phi_field.device) / (dx**2)
    stencil = stencil.view(1, 1, 3, 3, 3)
    phi_padded = F.pad(phi_field.unsqueeze(0).unsqueeze(0), (1,1,1,1,1,1), mode='circular')
    return F.conv3d(phi_padded, stencil, padding=0).squeeze(0).squeeze(0)

@torch.jit.script
def create_sponge_mask(N: int, sponge_width_frac: float, sponge_strength: float, device: torch.device) -> torch.Tensor:
    width = int(N * sponge_width_frac)
    ramp = torch.linspace(sponge_strength, 1.0, width, device=device)
    mask = torch.ones(N, N, N, device=device)
    mask[:width, :, :] *= ramp.view(-1, 1, 1); mask[-width:, :, :] *= ramp.view(-1, 1, 1).flip(0)
    mask[:, :width, :] *= ramp.view(1, -1, 1); mask[:, -width:, :] *= ramp.view(1, -1, 1).flip(1)
    mask[:, :, :width] *= ramp.view(1, 1, -1); mask[:, :, -width:] *= ramp.view(1, 1, -1).flip(2)
    return mask.to(torch.float16)

@torch.jit.script
def nlkg_derivative_structure(phi: torch.Tensor, phi_dot: torch.Tensor, dx: float, c_sq: float,
                              k_density: float, rho_thresh: float,
                              m_sq_vac: float, g_vac: float,
                              m_sq_part: float, g_part: float,
                              eta: float, alpha: float, delta: float) -> tuple[torch.Tensor, torch.Tensor]:
    phi_f32, phi_dot_f32 = phi.to(torch.float32), phi_dot.to(torch.float32)
    lap_phi = conv_laplacian_gpu(phi_f32, dx)

    # --- DENSITY-DEPENDENT PHYSICS ---
    rho = k_density * torch.pow(phi_f32, 2)
    particle_mask = (rho > rho_thresh).to(torch.float32)
    vacuum_mask = 1.0 - particle_mask

    m_sq_dynamic = particle_mask * m_sq_part + vacuum_mask * m_sq_vac
    g_dynamic = particle_mask * g_part + vacuum_mask * g_vac
    # --------------------------------

    potential_force = m_sq_dynamic * phi_f32 + g_dynamic * torch.pow(phi_f32, 3) + eta * torch.pow(phi_f32, 5)

    grad_phi_x = (torch.roll(phi_f32, -1, 0) - torch.roll(phi_f32, 1, 0)) / (2 * dx)
    grad_phi_y = (torch.roll(phi_f32, -1, 1) - torch.roll(phi_f32, 1, 1)) / (2 * dx)
    grad_phi_z = (torch.roll(phi_f32, -1, 2) - torch.roll(phi_f32, 1, 2)) / (2 * dx)
    grad_phi_abs_sq = grad_phi_x**2 + grad_phi_y**2 + grad_phi_z**2
    alpha_term = alpha * phi_f32 * phi_dot_f32 * grad_phi_abs_sq

    delta_term = delta * torch.pow(phi_dot_f32, 2) * phi_f32

    phi_ddot = c_sq * lap_phi - potential_force + alpha_term - delta_term
    return phi_dot, phi_ddot.to(phi.dtype)

@torch.jit.script
def update_phi_rk4_structure(phi_current: torch.Tensor, phi_dot_current: torch.Tensor, dt: float, args: list[torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor]:
    k1_v, k1_a = nlkg_derivative_structure(phi_current, phi_dot_current, *args)
    k2_v, k2_a = nlkg_derivative_structure(phi_current + 0.5*dt*k1_v, phi_dot_current + 0.5*dt*k1_a, *args)
    k3_v, k3_a = nlkg_derivative_structure(phi_current + 0.5*dt*k2_v, phi_dot_current + 0.5*dt*k2_a, *args)
    k4_v, k4_a = nlkg_derivative_structure(phi_current + dt*k3_v, phi_dot_current + dt*k3_a, *args)

    phi_next = phi_current + (dt / 6.0) * (k1_v + 2*k2_v + 2*k3_v + k4_v)
    phi_dot_next = phi_dot_current + (dt / 6.0) * (k1_a + 2*k2_a + 2*k3_a + k4_a)
    return phi_next, phi_dot_next

print("Structure Formation JIT functions defined (v9 - Final).")

In [ ]:
if __name__ == '__main__':
    print("--- INITIATING EFM STRUCTURE FORMATION SIMULATION (v9) ---")
    torch.manual_seed(42)

    phi_current = (torch.rand(config['N'], config['N'], config['N'], device=device) * config['initial_noise_amplitude']).to(torch.float16)
    phi_dot_current = torch.zeros_like(phi_current)

    sponge_mask = create_sponge_mask(config['N'], config['sponge_width'], config['sponge_strength'], device).to(torch.float16)

    history_size = config['T_steps'] // config['history_every_n_steps']
    max_phi_history = np.zeros(history_size)
    history_idx = 0

    pbar = tqdm(range(config['T_steps']), desc=f"Structure Formation ({config['N']}³)")
    sim_start_time = time.time()

    dt, dx, c_sq = config['dt_sim_unit'], config['dx_sim_unit'], config['c_sim_unit']**2

    for t_step in pbar:
        if t_step < config['transition_step']:
            # EPOCH 1: INFLATION (Simple tachyonic potential)
            k1_v, k1_a = phi_dot_current, c_sq * conv_laplacian_gpu(phi_current, dx) - config['m_sq_inflation'] * phi_current
            phi_next = phi_current + dt * k1_v
            phi_dot_next = phi_dot_current + dt * k1_a
        else:
            # EPOCH 3: STRUCTURE FORMATION (Full dynamic physics)
            if t_step == config['transition_step']:
                print("\n--- Reheating Transition: Activating Density-Dependent Physics ---")
            
            # Pack args for JIT function
            args = [
                dx, c_sq, config['k_density_coupling'], config['rho_threshold'],
                config['m_sq_vacuum'], config['g_vacuum'], config['m_sq_particle'], config['g_particle'],
                config['eta_sim'], config['alpha_structure'], config['delta_sim']
            ]
            phi_next, phi_dot_next = update_phi_rk4_structure(phi_current, phi_dot_current, dt, args)
        
        # Apply sponge and update fields
        phi_dot_next *= sponge_mask
        phi_current, phi_dot_current = phi_next, phi_dot_next
        
        if (t_step + 1) % config['history_every_n_steps'] == 0:
            if torch.any(torch.isinf(phi_current)) or torch.any(torch.isnan(phi_current)):
                print(f"\nERROR: NaN/Inf detected at step {t_step + 1}! Halting."); break
            max_phi = torch.max(torch.abs(phi_current)).item()
            if history_idx < history_size: max_phi_history[history_idx] = max_phi; history_idx += 1
            epoch_str = 'Inflation' if t_step < config['transition_step'] else 'Structure'
            pbar.set_postfix({'Max|φ|': f'{max_phi:.3e}', 'Epoch': epoch_str})

    else:
        sim_duration = time.time() - sim_start_time
        print(f"\nSimulation finished successfully in {sim_duration:.2f} seconds.")
        final_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        final_data_filename = os.path.join(data_path_dynamic, f"FINAL_DATA_{config['run_id']}_{final_timestamp}.npz")
        np.savez_compressed(final_data_filename, phi_final_cpu=phi_current.cpu().numpy(), config=config, max_phi_history=max_phi_history)
        print(f"Final simulation state saved to {final_data_filename}")

    del phi_current, phi_dot_current, sponge_mask; gc.collect(); torch.cuda.empty_cache()
    print("\n--- SIMULATION COMPLETE. ANALYSIS CAN NOW PROCEED. ---")

## Analysis of Self-Organized Structures (V9)

This analysis will visualize the final state. A successful run should show not just a vacuum, but distinct, high-amplitude, localized points (solitons) within that vacuum. We will also plot a histogram of the field's final values to look for a bimodal distribution, which would indicate a clear separation between the vacuum state and the soliton state.

In [ ]:
def analyze_structure_formation_results(data_file_path):
    print(f"\n--- Analyzing Structure Formation from {os.path.basename(data_file_path)} ---")
    try:
        with np.load(data_file_path, allow_pickle=True) as data:
            phi_final = data['phi_final_cpu'].astype(np.float32)
            config = data['config'].item()
            max_phi_history = data['max_phi_history']
    except FileNotFoundError:
        print(f"ERROR: Data file not found at {data_file_path}")
        return
    
    # Visualization
    N, center_slice = config['N'], config['N'] // 2
    vmax = np.percentile(np.abs(phi_final), 99.9) # Robust color scaling

    fig = plt.figure(figsize=(18, 18))
    gs = fig.add_gridspec(2, 2, height_ratios=[2, 1])

    ax1 = fig.add_subplot(gs[0, :])
    im1 = ax1.imshow(phi_final[center_slice, :, :], cmap='magma', origin='lower', vmin=-vmax, vmax=vmax)
    ax1.set_title(f'Final Field φ (slice z={center_slice}) - Note the Solitons', fontsize=16)
    fig.colorbar(im1, ax=ax1, orientation='vertical', fraction=0.046, pad=0.04)

    ax2 = fig.add_subplot(gs[1, 0])
    steps = (np.arange(len(max_phi_history)) + 1) * config['history_every_n_steps']
    ax2.plot(steps, max_phi_history)
    ax2.set_title('Maximum Field Amplitude |φ| Evolution', fontsize=16)
    ax2.set_xlabel('Time Step'); ax2.set_ylabel('Max |φ|'); ax2.set_yscale('log')
    ax2.grid(True, which='both', linestyle='--')
    ax2.axvline(x=config['transition_step'], color='r', linestyle='--', label=f'Reheating (t={config["transition_step"]})')
    ax2.legend()

    ax3 = fig.add_subplot(gs[1, 1])
    ax3.hist(phi_final.flatten(), bins=200, log=True, range=(-vmax, vmax))
    ax3.set_title('Histogram of Final Field Values (Log Scale)', fontsize=16)
    ax3.set_xlabel('φ value'); ax3.set_ylabel('Count')

    fig.suptitle(f"EFM Structure Formation Results ({config['run_id']})", fontsize=22, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(os.path.join(data_path_dynamic, f"analysis_{config['run_id']}.png"))
    plt.show()

if __name__ == '__main__':
    data_files = glob.glob(os.path.join(data_path_dynamic, '*.npz'))
    if data_files:
        latest_file = max(data_files, key=os.path.getctime)
        analyze_structure_formation_results(latest_file)
    else:
        print("\nAnalysis skipped. No data file found.")